In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Osteoarthritis"
cohort = "GSE141934"

# Input paths
in_trait_dir = "../../input/GEO/Osteoarthritis"
in_cohort_dir = "../../input/GEO/Osteoarthritis/GSE141934"

# Output paths
out_data_file = "../../output/preprocess/Osteoarthritis/GSE141934.csv"
out_gene_data_file = "../../output/preprocess/Osteoarthritis/gene_data/GSE141934.csv"
out_clinical_data_file = "../../output/preprocess/Osteoarthritis/clinical_data/GSE141934.csv"
json_path = "../../output/preprocess/Osteoarthritis/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Transcriptional data of inflamatory arthritis T cells."
!Series_summary	"With a focus on rheumatoid arthritis (RA), we sought new insight into genetic mechanisms of adaptive immune dysregulation to help prioritise molecular pathways for targeting in this and related immune pathologies. Whole genome methylation and transcriptional data from isolated CD4+ T cells and B cells of >100 genotyped and phenotyped inflammatory arthritis patients, all of whom were naïve to immunomodulatory treatments, were obtained. Analysis integrated these comprehensive data with GWAS findings across IMDs and other publically available resources."
!Series_overall_design	"Suspected inflammatory arthritis patients of Northern European ancestry were recruited prior to treatment with immunomodulatory drugs. RA patients were classified using current, internationally accepted criteria, and matched with disease controls in respect of demographic and clinical characteristics. CD4

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the background description, this dataset contains transcriptional data 
# from CD4+ T cells, which indicates gene expression data is available
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# For trait (Osteoarthritis):
# From the sample characteristics, we can see the disease status in the 'working_diagnosis' field (row 6)
# This includes 'Osteoarthritis' which is our trait of interest
trait_row = 6

# For age:
# Age is clearly available in row 2
age_row = 2

# For gender:
# Gender is available in row 1
gender_row = 1

# 2.2 Data Type Conversion

def convert_trait(value):
    """Convert working_diagnosis to binary (0 or 1) based on Osteoarthritis status."""
    if pd.isna(value) or not isinstance(value, str):
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # 1 if the diagnosis is Osteoarthritis, 0 otherwise
    if value.lower() == 'osteoarthritis':
        return 1
    else:
        return 0

def convert_age(value):
    """Convert age to continuous numeric value."""
    if pd.isna(value) or not isinstance(value, str):
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    try:
        return float(value)
    except (ValueError, TypeError):
        return None

def convert_gender(value):
    """Convert gender to binary (0 for female, 1 for male)."""
    if pd.isna(value) or not isinstance(value, str):
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if value.upper() == 'F':
        return 0
    elif value.upper() == 'M':
        return 1
    else:
        return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Validate and save cohort info (initial filtering)
is_usable = validate_and_save_cohort_info(
    is_final=False, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available, 
    is_trait_available=is_trait_available
)

# We'll need to wait for the next step where we should get the actual clinical data
# For now, let's just inform about the identified rows and conversions
print(f"Trait row identified: {trait_row}")
print(f"Age row identified: {age_row}")
print(f"Gender row identified: {gender_row}")
print(f"Gene data available: {is_gene_available}")
print(f"Trait data available: {is_trait_available}")
print("Conversion functions defined for trait, age, and gender")


Trait row identified: 6
Age row identified: 2
Gender row identified: 1
Gene data available: True
Trait data available: True
Conversion functions defined for trait, age, and gender


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Osteoarthritis/GSE141934/GSE141934_series_matrix.txt.gz
Gene data shape: (11710, 100)
First 20 gene/probe identifiers:
Index(['ILMN_1343291', 'ILMN_1343295', 'ILMN_1651228', 'ILMN_1651229',
       'ILMN_1651254', 'ILMN_1651259', 'ILMN_1651262', 'ILMN_1651278',
       'ILMN_1651315', 'ILMN_1651316', 'ILMN_1651341', 'ILMN_1651343',
       'ILMN_1651347', 'ILMN_1651378', 'ILMN_1651385', 'ILMN_1651403',
       'ILMN_1651405', 'ILMN_1651429', 'ILMN_1651433', 'ILMN_1651438'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Analyzing the gene identifiers
# The IDs with "ILMN_" prefix are Illumina probe IDs, not human gene symbols
# These Illumina probe IDs need to be mapped to human gene symbols for biological interpretation

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Let's look for platform information in the SOFT file to understand the annotation better
print("\nSearching for platform information in SOFT file:")
with gzip.open(soft_file, 'rt') as f:
    for i, line in enumerate(f):
        if '!Series_platform_id' in line:
            print(line.strip())
            break
        if i > 100:  # Limit search to first 100 lines
            print("Platform ID not found in first 100 lines")
            break

# Check if the SOFT file includes any reference to gene symbols
print("\nSearching for gene symbol information in SOFT file:")
with gzip.open(soft_file, 'rt') as f:
    gene_symbol_lines = []
    for i, line in enumerate(f):
        if 'GENE_SYMBOL' in line or 'gene_symbol' in line.lower() or 'symbol' in line.lower():
            gene_symbol_lines.append(line.strip())
        if i > 1000 and len(gene_symbol_lines) > 0:  # Limit search but ensure we found something
            break
    
    if gene_symbol_lines:
        print("Found references to gene symbols:")
        for line in gene_symbol_lines[:5]:  # Show just first 5 matches
            print(line)
    else:
        print("No explicit gene symbol references found in first 1000 lines")

# Look for alternative annotation files or references in the directory
print("\nChecking for additional annotation files in the directory:")
all_files = os.listdir(in_cohort_dir)
print([f for f in all_files if 'annotation' in f.lower() or 'platform' in f.lower() or 'gpl' in f.lower()])



Gene annotation preview:
Columns in gene annotation: ['ID', 'Species', 'Source', 'Search_Key', 'Transcript', 'ILMN_Gene', 'Source_Reference_ID', 'RefSeq_ID', 'Unigene_ID', 'Entrez_Gene_ID', 'GI', 'Accession', 'Symbol', 'Protein_Product', 'Probe_Id', 'Array_Address_Id', 'Probe_Type', 'Probe_Start', 'SEQUENCE', 'Chromosome', 'Probe_Chr_Orientation', 'Probe_Coordinates', 'Cytoband', 'Definition', 'Ontology_Component', 'Ontology_Process', 'Ontology_Function', 'Synonyms', 'Obsolete_Probe_Id', 'GB_ACC']
{'ID': ['ILMN_1343048', 'ILMN_1343049', 'ILMN_1343050', 'ILMN_1343052', 'ILMN_1343059'], 'Species': [nan, nan, nan, nan, nan], 'Source': [nan, nan, nan, nan, nan], 'Search_Key': [nan, nan, nan, nan, nan], 'Transcript': [nan, nan, nan, nan, nan], 'ILMN_Gene': [nan, nan, nan, nan, nan], 'Source_Reference_ID': [nan, nan, nan, nan, nan], 'RefSeq_ID': [nan, nan, nan, nan, nan], 'Unigene_ID': [nan, nan, nan, nan, nan], 'Entrez_Gene_ID': [nan, nan, nan, nan, nan], 'GI': [nan, nan, nan, nan, nan], '

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine which columns contain probe IDs and gene symbols
# From the gene annotation preview, we can see:
# - 'ID' column contains the probe identifiers (e.g., ILMN_1343048)
# - 'Symbol' column contains the gene symbols

# 2. Create a gene mapping dataframe
mapping_df = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Symbol')
print(f"Gene mapping dataframe shape: {mapping_df.shape}")
print("Sample of gene mapping data:")
print(mapping_df.head())

# 3. Apply gene mapping to convert probe-level data to gene-level expression data
gene_data = apply_gene_mapping(gene_data, mapping_df)
print(f"Gene expression data shape after mapping: {gene_data.shape}")
print("First few rows of gene expression data:")
print(gene_data.head())

# 4. Clean and normalize gene symbols in the index
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene expression data shape after normalization: {gene_data.shape}")
print("First few gene symbols after normalization:")
print(gene_data.index[:10])

# Save the gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Gene mapping dataframe shape: (44837, 2)
Sample of gene mapping data:
             ID                     Gene
0  ILMN_1343048      phage_lambda_genome
1  ILMN_1343049      phage_lambda_genome
2  ILMN_1343050  phage_lambda_genome:low
3  ILMN_1343052  phage_lambda_genome:low
4  ILMN_1343059                     thrB
Gene expression data shape after mapping: (8340, 100)
First few rows of gene expression data:
         GSM4216498  GSM4216499  GSM4216500  GSM4216501  GSM4216502  \
Gene                                                                  
A2LD1      6.624761    6.623393    6.723056    6.903384    6.048768   
AACS       5.615109    5.192011    5.095448    5.465379    5.257583   
AADACL1   13.308954   12.461564   12.518904   13.477049   13.164655   
AAGAB      8.607497    8.284817    8.830837    8.128734    8.351699   
AAK1       9.258350    8.017645    9.570424    7.815827   10.240408   

         GSM4216503  GSM4216504  GSM4216505  GSM4216506  GSM4216507  ...  \
Gene            

Gene expression data saved to ../../output/preprocess/Osteoarthritis/gene_data/GSE141934.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data 
# (This was already done in the previous step, so no need to repeat)
print(f"Normalized gene data shape: {gene_data.shape}")

# 2. Examine the sample IDs in the gene expression data to understand the structure
print("Gene data column names (sample IDs):")
print(gene_data.columns[:5])  # Print first 5 for brevity

# Inspect the clinical data format from the matrix file directly
background_info, clinical_data = get_background_and_clinical_data(matrix_file)
print("\nRaw clinical data structure:")
print(f"Clinical data shape: {clinical_data.shape}")
print(f"Clinical data columns: {clinical_data.columns[:5]}")

# Get the sample characteristics to re-extract the disease information
sample_characteristics_dict = get_unique_values_by_row(clinical_data)
print("\nSample characteristics dictionary:")
print(sample_characteristics_dict)

# 3. Directly create clinical features from the raw data again
# Verify trait row contains the disease information (OA vs RA)
print("\nValues in trait row:")
trait_values = clinical_data.iloc[trait_row].values
print(trait_values[:5])

# Create clinical dataframe with proper structure
# First get the sample IDs from gene data as these are our actual sample identifiers
sample_ids = gene_data.columns.tolist()

# Create the clinical features dataframe with those sample IDs
clinical_features = pd.DataFrame(index=[trait], columns=sample_ids)

# Fill the clinical features with our trait values by mapping GSM IDs to actual values
for col in clinical_data.columns:
    if col in sample_ids:
        # Extract the disease value and convert it
        disease_val = clinical_data.iloc[trait_row][col]
        clinical_features.loc[trait, col] = convert_trait(disease_val)

print("\nCreated clinical features dataframe:")
print(f"Shape: {clinical_features.shape}")
print(clinical_features.iloc[:, :5])  # Show first 5 columns

# 4. Link clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_features, gene_data)
print(f"\nLinked data shape before handling missing values: {linked_data.shape}")

# 5. Handle missing values - we need to use the actual column name, not the trait variable
# First identify the actual trait column name in the linked data
trait_column = clinical_features.index[0]  # This should be 'Osteoarthritis'
print(f"Actual trait column in linked data: {trait_column}")

# Now handle missing values with the correct column name
linked_data_clean = handle_missing_values(linked_data, trait_column)
print(f"Linked data shape after handling missing values: {linked_data_clean.shape}")

# 6. Evaluate bias in trait and demographic features
is_biased, linked_data_clean = judge_and_remove_biased_features(linked_data_clean, trait_column)

# 7. Conduct final quality validation
note = "Dataset contains gene expression data from synovial fibroblasts of RA and OA patients. Data includes high serum and low serum responses."
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=(linked_data_clean.shape[0] > 0),
    is_biased=is_biased,
    df=linked_data_clean,
    note=note
)

# 8. Save linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data_clean.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset deemed not usable due to quality issues - linked data not saved")

Normalized gene data shape: (8143, 100)
Gene data column names (sample IDs):
Index(['GSM4216498', 'GSM4216499', 'GSM4216500', 'GSM4216501', 'GSM4216502'], dtype='object')

Raw clinical data structure:
Clinical data shape: (7, 101)
Clinical data columns: Index(['!Sample_geo_accession', 'GSM4216498', 'GSM4216499', 'GSM4216500',
       'GSM4216501'],
      dtype='object')

Sample characteristics dictionary:
{0: ['patient: 1072', 'patient: 1085', 'patient: 1076', 'patient: 1087', 'patient: 1080', 'patient: 1088', 'patient: 1083', 'patient: 1094', 'patient: 1050', 'patient: 1067', 'patient: 1051', 'patient: 1054', 'patient: 1070', 'patient: 1058', 'patient: 2010', 'patient: 2012', 'patient: 2029', 'patient: 2075', 'patient: 2062', 'patient: 2078', 'patient: 2086', 'patient: 2087', 'patient: 2067', 'patient: 2072', 'patient: 2090', 'patient: 1019', 'patient: 1020', 'patient: 1003', 'patient: 1008', 'patient: 2030'], 1: ['gender: F', 'gender: M'], 2: ['age: 50', 'age: 43', 'age: 66', 'age: 55

/media/techt/DATA/GenoAgent/tools/preprocess.py:455: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[gene_cols] = df[gene_cols].fillna(df[gene_cols].mean())


Linked data shape after handling missing values: (100, 8144)
For the feature 'Osteoarthritis', the least common label is '1' with 5 occurrences. This represents 5.00% of the dataset.
The distribution of the feature 'Osteoarthritis' in this dataset is fine.



Linked data saved to ../../output/preprocess/Osteoarthritis/GSE141934.csv
